In [2]:
from flashtext import KeywordProcessor
import os
import re
import collections
import pandas as pd
import uszipcode
import zipcode
import us
import pickle
import string
import timeit
os.environ['ZIPCODE_CONNECTION_STRING'] = 'sqlite:////home/modp/zipcode.db'

ads_file = 'us-ads.csv'
cities_file = 'us-cities/uscities.csv'
all_regions_list = pickle.load(open( "all_regions.p", "rb" ) )

county_fips_file = 'geo-data/county_fips.csv'
cg_fips_file = 'geo-data/US_Congress_FIPS.csv'

# fips_file = 'geo-data/ZIP-COUNTY-FIPS_2018-03.csv'
fips_file = 'US_FIPS.csv'
district_fips_file = 'district-fips.csv'

# County,State,USA -> County, State
county_st_us = re.compile('([A-Za-z]+\sCounty),(\w{2}),(USA)')

county_state_us_regex = re.compile(r'\b([A-Za-z]+\sCounty),(\w{2}),USA\b')
county_regex = re.compile(r'\b([A-Za-z]+\s?[A-Za-z]+)(\sCounty)\b')
city_regex = re.compile(r'\b([A-Za-z]+\s?[A-Za-z]{3,})\b')
city_state_regex = re.compile(r'\b([A-Za-z]+\s?[A-Za-z]{3,}),\s+([A-Z]{2})\b')

district_regex= re.compile(r'\b([A-Z]{2}-\d{1,2})(,[A-Z]{2},USA)\b')

cg_regex = re.compile(r'\b([A-Z]{2})(-)(\d{1,2})\b')
only_zips = re.compile(r'\b^[0-9]+|,$\b')
zip_regex = re.compile(r'\b\d{5}\b')
state_regex = re.compile(r'\b[A-Z]{2}\b')
geotarget = 'Geo_Targeting_Included'


In [6]:
# district_fips.drop('Unnamed: 0', axis=1, inplace=True)
# district_fips.to_csv('district-fips.csv', index=False)

fips = pd.read_csv(fips_file, dtype=str)
district_fips = pd.read_csv('district-fips.csv', dtype=str)


In [21]:
def cleanup_old(df):
    # Change 'United States' to 'USA', abbreviate state names
    for idx,row in  df.iterrows():
        locations = row[geotarget]
        # Create two sets 
        # First set catches { Arizona, Georgia, Idaho, ... }
        # Second set catches { AZ-1,Arizona,United States, ... }
        state_matches = us_states & set(locations.split(',')) | us_states & set(locations.split(', '))
        for match in state_matches:
            locations = locations.replace(match, state2abbr[match])
#         locations = locations.replace(re.escape('WA, DC (Hagerstown, Maryland), WA'), "Hangerstown, MD")
        df.at[idx, geotarget] = locations

    # Simplify district name information
    # e.g: AZ-1,AZ,USA -> AZ-1
    df[geotarget] = df[geotarget].str.replace(district_regex, r'\1')
    df.Geo_Targeting_Included = df.Geo_Targeting_Included.str.replace(county_st_us,r'\1, \2' )

    special_cases = {
        'George s': "George's",
        'Raleigh-Durham (Fayetteville), NC' : 'Fayetteville, NC',
        'WA, DC (Hagerstown, Maryland), WA' : "" ,
        'WA, DC (Hagerstown, MD), WA' : "Hagerstown, MD" ,
        'Richmond-Petersburg, VA' : 'Richmond, VA, Petersburg, VA',
        'Harrisburg-Lancaster-York, PA' : 'Cumberland County, Dauphin County, Perry County, Lebanon County, Adams County, York County'
    }
    for case in special_cases:
        df[geotarget] = df[geotarget].str.replace(re.escape(case), special_cases[case])
    df[geotarget] = df[geotarget].str.replace('George s',"George's")
   
    return df

def cleanup_states(data):
    state_matches = us_states & set(data.split(',')) | us_states & set(data.split(', '))
    for match in state_matches:
        data = data.replace(match, state2abbr[match])   
    return data 

def cleanup(data):
    # Change 'United States' to 'USA', abbreviate state names
        # Create two sets 
        # First set catches { Arizona, Georgia, Idaho, ... }
        # Second set catches { AZ-1,Arizona,United States, ... }
        
#         state_matches = us_states & set(data.split(',')) | us_states & set(data.split(', '))
#         for match in state_matches:
#             data = data.replace(match, state2abbr[match])


    # Simplify district name information
    # e.g: AZ-1,AZ,USA -> AZ-1
    data[geotarget] = data[geotarget].str.replace(district_regex, r'\1')
    data[geotarget] = data[geotarget].str.replace(county_st_us,r'\1, \2' )

    special_cases = {
        'George s': "George's",
        'Raleigh-Durham (Fayetteville), NC' : 'Fayetteville, NC',
        'WA, DC (Hagerstown, Maryland), WA' : "" ,
        'WA, DC (Hagerstown, MD), WA' : "Hagerstown, MD" ,
        'Richmond-Petersburg, VA' : 'Richmond, VA, Petersburg, VA',
        'Harrisburg-Lancaster-York, PA' : 'Cumberland County, Dauphin County, Perry County, Lebanon County, Adams County, York County'
    }
    for case in special_cases:
        data[geotarget] = data[geotarget].str.replace(re.escape(case), special_cases[case])
#     df[geotarget] = df[geotarget].str.replace('George s',"George's")
    return data


def make_state_dict():
    """
    Make dictionaries that map state names and abbreviations
    e.g. Iowa -> IA, IA -> Iowa
    :return:
    """
    state2abbr = {}
    abbr2state = {}
    for state in us.states.STATES:
        state2abbr[state.name] = state.abbr
        abbr2state[state.abbr] = state.name
    state2abbr['United States'] = 'USA'
    abbr2state['US'] = 'United States'
    return state2abbr, abbr2state

state2abbr, abbr2state = make_state_dict()
us_states = [s.name for s in us.states.STATES]
us_states = set(us_states)
us_states.add('United States')
c_districts = pickle.load(open( "cg_zips.p", "rb" ) )

In [9]:
import sqlite3
conn = sqlite3.connect('/home/modp/zipcode.db')
cursor = conn.cursor()

# cursor.description
cursor.execute('''SELECT DISTINCT city, state from zipcodes''')
city_state = cursor.fetchall()

cursor.execute('''SELECT county from zipcodes''')
counties = cursor.fetchall()

cursor.execute('''SELECT county, state from zipcodes''')
counties_state = cursor.fetchall()

cursor.execute('''SELECT DISTINCT city from zipcodes''')
cities = cursor.fetchall()

cursor.execute('''SELECT state from zipcodes''')
states = cursor.fetchall()

cursor.execute('''SELECT zipcode from zipcodes''')
all_zipcodes = cursor.fetchall()


city_state_set = [', '.join(c) for c in city_state]
counties_set  = [', '.join(c) for c in counties]
counties_state_set  = [', '.join(c) for c in counties_state]
zipcodes_set = [', '.join(z) for z in all_zipcodes]
cities_set = [', '.join(c) for c in cities]
state_abbr_set = [', '.join(s) for s in states]
state_names_set = [state.name for state in us.states.STATES]

district_regions_set = []
for cd in c_districts:
    for d in c_districts[cd]: 
        district_regions_set.append(cd + '-' +str(d))

# all_regions = city_state_set + counties_set + counties_state_set + cities_set + state_names_set + state_abbr_set + district_regions_set
# pickle.dump( all_regions, open( "all_regions.p", "wb" ) )
conn.close()

# Create a dictionary mapping regions to FIPS

In [10]:
# Create a CITYSTATE column e.g. Austin, TX
fips['CITYSTATE'] = fips['CITY'] + ', ' + fips['STATE']

# Create a COUNTRYSTATE column: e.g. King's County, NY
fips['COUNTYSTATE'] = fips['COUNTY'] + ', ' + fips['STATE']

# County/State dictionary 
fips_countystate = fips.set_index('COUNTYSTATE')
countystate_dict = fips_countystate['FIPS'].to_dict()

# City/State dictionary 
fips_city = fips.set_index('CITYSTATE')
citystate_dict = fips_city['FIPS'].to_dict()

# Zipcode dictionary
f = fips.set_index('ZIP')
zip2fips = f['FIPS'].to_dict()

# District dictionary
d = district_fips.set_index('DISTRICT')
district2fips = d['FIPS'].to_dict()

# Create dict for counties e.g. state_fips['King's County'] = [fips1, fips2, ... ]
e = fips[['COUNTY', 'FIPS']].set_index('COUNTY')
county_fips = e.groupby('COUNTY').apply(lambda dfg: dfg.to_dict(orient='list')).to_dict()
new_f = {}
for county in county_fips:
    new_f[county] = list(set(county_fips[county]['FIPS']))
county2fips = new_f 

# Create dict for states e.g. state_fips['AK'] = [fips1, fips2, ... ]
d = fips[['STATE', 'FIPS']].set_index('STATE')
state_fips = d.groupby('STATE').apply(lambda dfg: dfg.to_dict(orient='list')).to_dict()
new_f = {}
for state in state_fips:
    new_f[state] = list(set(state_fips[state]['FIPS']))
state2fips = new_f    


# Merge all dictionaries together 
fips_dict = {**state2fips, **county2fips, **district2fips, **zip2fips, **citystate_dict, **countystate_dict}

# Keywords

Create three keyword processors. One for just cities, one for states, and one for everything else. The idea is to sift through the geo-targeting data and remove everything but lone cities and states first.   

In [11]:
# Create keyword processor for everything but the 
kwd = KeywordProcessor()
kwd_cities = KeywordProcessor()
kwd_states = KeywordProcessor()


# for cg_dist in district_fips.DISTRICT:
#     kwd.add_keyword(city_state, ('District', cg_dist))

for dist in district_regions_set:
    kwd.add_keyword(dist, ('District', dist))
    
for county in counties_set:
    kwd.add_keyword(county, ('County', county))
    
for county in counties_state_set:
    kwd.add_keyword(county, ('County', county))
    
    
for z in zipcodes_set:
    kwd.add_keyword(z, ('Zipcode', z))
    
for city_state in city_state_set:
    kwd.add_keyword(city_state, ('City/State', city_state))
    
for city in cities_set:
    kwd_cities.add_keyword(city, ('City', city))
    
for state in state_abbr_set:
    kwd_states.add_keyword(state, ('State', state))
    
kwd_replace = KeywordProcessor()
for k in kwd.get_all_keywords().keys():
    kwd_replace.add_keyword(k, ' ')    

    

In [12]:
# Handle the keyword extraction, return FIPS 
def handler(data):
    # Ignoring ads targeted to all of USA
    if data == 'USA':
        return data
    
    f = set()
    
    # Get the first batch of keywords
    # City/State, District, County
    keys = kwd.extract_keywords(data)
    
    # Remove keywords already found  
    data = kwd_replace.replace_keywords(data)
    # Get keywords of only cities 
    keys += kwd_cities.extract_keywords(data)
    # Get keywords of only states
    keys += kwd_states.extract_keywords(data)
    
    for key in keys:
        res = get_fips(key)
        if type(res) == list:
            f |= set(res)
        else:
            f.add(res)
            
    if None in f:
        f.remove(None)
    
    # Return as a string delimited by comma
    new_string = ', '.join(f)
    return new_string

def get_fips(value):
    """
    If zipcode isn't in the list, check the zipcode package
    and get the county name - lookup FIPS of county name
    """
    if value[0] == 'Zipcode':
        fips = fips_dict.get(value[1])
        if not fips:
            county = zipcode.isequal(value[1]).county
            return fips_dict.get(county)
    
    # Ignoring cities for now - its ambiguous
    if value[0] == 'City':
        return value[1]
    
    return fips_dict.get(value[1])
    

In [26]:
import time

df1 = pd.read_csv(ads_file)
# df1 = pd.read_csv('ads_file_cleaned', nrows=100)

start = time.time()

df1[geotarget] = df1[geotarget].apply(cleanup_states)
df1 = cleanup(df1)
end = time.time()
print("Time spent cleaning up: ",end - start)

start = time.time()
df1[geotarget] = df1[geotarget].apply(handler)
end = time.time()
print("Time spent parsing: ",end - start)
# df1.to_csv('Ads-Filtered.csv', index=False)

Time spent cleaning up:  14.37647533416748
Time spent parsing:  155.04773712158203


In [27]:
df1.to_csv('Ads-Filtered.csv', index=False)

In [28]:
df1.to_csv('Ads-Filtered.csv', index=False)